In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np

from datetime import datetime, timedelta, timezone

import sys
sys.path.append('../')
from utils import data
from models.resnet import OctResnet18
from utils import visualization

In [2]:
# const
taskName = 'fasionClassfiction'
modelName = 'resnet18_bn'
JST = timezone(timedelta(hours=+9), 'JST')
now = datetime.now(JST)
nowStr = now.strftime("%Y%m%d_%H%M%S")
showImgCount = 10

imgHeight = 224
imgWidth = 224
imgChannel = 3
imgShape = [-1, imgHeight, imgWidth, imgChannel]
labelSize = 13
tfrecordPath = './img/fashionDataset/tfrecord/dataset224.tfrecord'
trainRatio = 0.7

# hyper parameter
bs = 64
lr = 0.0001
ep = 50
alpha = 0.25

logDir = '../logs/{}/{}/{}/'.format(taskName, modelName, nowStr)
metadataDir = '{}metadata.tsv'.format(logDir)
checkPointDir = '{}images.ckpt'.format(logDir)

In [3]:
# main
with tf.Graph().as_default():
    # load data
    with tf.variable_scope('tfrecord'):
        tfrecord = data.TFRecord(tfrecordPath, labelSize)
        dataset = tfrecord.toDataset()
        # split dataset
        tfrecord = data.TFRecord(tfrecordPath, labelSize)
        dataset = tfrecord.toDataset()
        trainSize, testSize, trainDataset, testDataset = tfrecord.splitDataset(bs)
        trainIteration = trainSize // bs
        testIteration = testSize // bs        
        # TODO: data augmentation

    # make iterator
    with tf.variable_scope('train_data'):
        trainIterator = trainDataset.make_initializable_iterator()
        trainNextBatch = trainIterator.get_next(name='train_next_batch')
        trainIteratorInitOp = trainIterator.initializer
    with tf.variable_scope('test_data'):
        testIterator = testDataset.make_initializable_iterator()
        testNextBatch = testIterator.get_next(name='test_next_batch')
        testIteratorInitOp = testIterator.initializer

    
    x = tf.placeholder("float", [None, imgHeight, imgWidth, imgChannel], name='x')
    y = tf.placeholder("float", [None, labelSize], name='label')
    isTrain = tf.placeholder(tf.bool, name='is_train')
    
    network = OctResnet18(x, labelSize=labelSize, alpha=alpha)
    _ = network.inference(isTrain)
    cost = network.loss(y)
    optimizer = tf.train.AdamOptimizer(lr)
    accuracy = network.accuracy(y)
    trainOp = network.training(cost, optimizer)

    # tensor board
    with tf.variable_scope('train'):
        drawTrainAcc = tf.summary.scalar('acc', accuracy)
        drawTrainLoss = tf.summary.scalar('loss', cost)
        trainSummaryOp = tf.summary.merge([drawTrainAcc, drawTrainLoss])
    with tf.variable_scope("validation"):
        drawValAcc = tf.summary.scalar('acc', accuracy)
        drawValLoss = tf.summary.scalar('loss', cost)
        valSummaryOp = tf.summary.merge([drawValAcc, drawValLoss])
    
    drawImage = tf.summary.image('train_images', tf.reshape(x, imgShape), showImgCount)
    allVariables = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)
    summaryOp = tf.summary.merge([drawImage])
    
    saver = tf.train.Saver()
    config = tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))
    sess = tf.Session(config=config)
    summaryWriter = tf.summary.FileWriter(logDir, graph=sess.graph)

    sess.run(tf.global_variables_initializer())

    # Training cycle
    for e in range(ep):
        # train
        sess.run(trainIteratorInitOp)
        for i in range(trainIteration):
            _, batchX, batchY = sess.run(trainNextBatch)
            sess.run(trainOp, feed_dict={x: batchX, y: batchY, isTrain: True})
        # Display logs per epoch step
        # TODO: coculate avg
        trainSummary = sess.run([trainSummaryOp,summaryOp], feed_dict={x: batchX, y: batchY, isTrain: False})
        for summary in trainSummary:
            summaryWriter.add_summary(summary, e)
        # validation
        sess.run(testIteratorInitOp)
        for i in range(testIteration):
            _, testBatchX, testBatchY = sess.run(testNextBatch)
            sess.run([cost, accuracy], feed_dict={x: testBatchX, y: testBatchY, isTrain: False})
        # TODO: coculate avg
        valSummary = sess.run([valSummaryOp], feed_dict={x: testBatchX, y: testBatchY, isTrain: False})
        for summary in valSummary:
            summaryWriter.add_summary(summary, e)
            
        #saver.save(sess, checkPointDir, global_step=e)
        trainAcc = sess.run(accuracy, feed_dict={x: batchX, y: batchY, isTrain: False})
        valAcc = sess.run(accuracy, feed_dict={x: testBatchX, y: testBatchY, isTrain: False})
        print('Epoch: {} | Train Accuracy: {} | Validation Accuracy: {}'.format(e, trainAcc, valAcc))
    summaryWriter.close()
    sess.close()

(?, 56, 56, 48) (?, 28, 28, 16) (?, 56, 56, 48) (?, 28, 28, 16)
(?, 56, 56, 48) (?, 28, 28, 16) (?, 56, 56, 48) (?, 28, 28, 16)
(?, 28, 28, 96) (?, 14, 14, 32) (?, 28, 28, 96) (?, 14, 14, 32)
(?, 28, 28, 96) (?, 14, 14, 32) (?, 28, 28, 96) (?, 14, 14, 32)
(?, 14, 14, 192) (?, 7, 7, 64) (?, 14, 14, 192) (?, 7, 7, 64)
(?, 14, 14, 192) (?, 7, 7, 64) (?, 14, 14, 192) (?, 7, 7, 64)


ValueError: Dimensions must be equal, but are 7 and 8 for 'stage4/residualBlock1/conv1/add' (op: 'Add') with input shapes: [?,7,7,384], [?,8,8,384].